# ㅇㅇ

In [43]:
import os
import json
import uuid
import logging
import fitz  # PyMuPDF

from typing import List, Dict, Any, Optional
from dotenv import load_dotenv
from tqdm import tqdm
from openai import OpenAI

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

LLM_MODEL_FOR_PARSING = "gpt-4o"        
LLM_MODEL_FOR_DECOMPOSITION = "gpt-4o" 

In [46]:
## RFP에서 요구사항 추출 및 정제
def extract_text_with_page_info(pdf_path):
    """
    PDF 파일에서 텍스트를 페이지별로 추출하고, 각 페이지 시작에 페이지 번호를 표기합니다.
    (사용자 스크립트에 이미 정의된 함수로 가정)
    """
    document = fitz.open(pdf_path)
    full_text_with_pages = []
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text = page.get_text()
        # 페이지 마커를 맨 앞에 추가하고, 실제 텍스트 내용과 명확히 구분되도록 줄바꿈 추가
        full_text_with_pages.append(f"---PAGE_START_{page_num+1}---\n{text.strip()}")
    return "\n".join(full_text_with_pages), document.page_count

def initial_text_split(text, chunk_size=4000, chunk_overlap=200):
    """
    텍스트를 LLM이 처리할 수 있는 크기로 초기 분할합니다.
    (사용자 스크립트에 이미 정의된 함수로 가정)
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n---PAGE_START_\d+---\n", "\n\n", "\n", ". ", "? ", "! ", " ", ""],
        keep_separator=True # 페이지 마커 유지를 위해 True 권장
    )
    docs = text_splitter.create_documents([text])
    return docs

def llm_based_semantic_chunking_for_dev_reqs(initial_chunks, client_instance, llm_model="gpt-4o"): # client_instance로 명칭 변경
    final_semantic_chunks = []
    req_id_counter = 0

    system_prompt = """
    당신은 RFP(제안요청서) 문서에서 시스템 요구사항을 전문적으로 분석하고 개발 표준에 맞춰 구조화하는 시니어 비즈니스 분석가입니다.
    사용자가 제공하는 텍스트는 RFP 문서의 특정 섹션이며, 여기서 모든 기능적, 비기능적, 성능, 보안, 데이터, 제약사항 요구사항을 추출해야 합니다.

    각 요구사항은 다음 속성을 포함하는 JSON 객체로 정의되어야 합니다.

    -   **id**: 고유 식별자. 예를 들어, 기능적 요구사항은 "FUNC-001", 비기능적은 "NFR-001", 보안은 "SEC-001"과 같이 유형 접두사와 일련번호를 결합하여 생성하십시오. (RFP 원문에 ID가 있다면 그것을 우선 사용)
    -   **type**: 요구사항의 유형. 다음 중 하나를 선택: "기능적", "비기능적", "성능", "보안", "데이터", "제약사항", "시스템 장비 구성", "컨설팅", "테스트", "품질", "프로젝트 관리", "프로젝트 지원", "기타". 텍스트에 명시된 내용에 따라 가장 적합한 유형을 선택하십시오. (예: "시스템 장비 구성요구사항", "컨설팅 요구사항" 등 PDF의 분류명 활용)
    -   **description**: 요구사항에 대한 명확하고 간결한 설명. 원본 PDF의 '요구사항 명칭'과 '정의', '상세설명/세부내용'을 종합하여 개발 친화적인 형태로 작성하십시오. 하나의 요구사항은 하나의 독립적인 기능 또는 특성을 나타내야 합니다.
    -   **acceptance_criteria**: 이 요구사항이 충족되었음을 검증할 수 있는 구체적인 테스트 조건이나 결과. 1~2개의 명확한 문장으로 서술하십시오. 원본 텍스트에 직접적인 검증 기준이 없더라도, 요구사항의 내용에 기반하여 합리적으로 추론하여 작성하십시오. (예: "사용자가 [기능]을 수행했을 때, [예상 결과]가 나타난다.")
    -   **priority**: 요구사항의 중요도. 다음 중 하나를 선택: "필수", "높음", "중간", "낮음". (텍스트에 명시되어 있다면 그대로 사용, 아니면 일반적으로 "필수"로 추정)
    -   **responsible_module**: 이 요구사항이 주로 영향을 미치거나 구현될 것으로 예상되는 시스템/애플리케이션의 주요 모듈 또는 영역 (예: "로그인 모듈", "결제 시스템", "관리자 페이지", "데이터베이스"). 텍스트 내용을 기반으로 추론하십시오.
    -   **source_pages**: 이 요구사항이 발견된 원본 PDF 페이지 번호 리스트. 페이지 구분자(`---PAGE_START_N---`)를 참조하여 정확한 페이지 번호를 파싱하십시오.
    -   **raw_text_snippet**: 이 요구사항이 포함된 원본 텍스트 스니펫. 해당 요구사항을 추출하는 데 사용된 원본 문장 또는 단락(예: 표의 해당 행 전체 내용)을 포함하십시오.

    응답은 **JSON 배열 형식으로만** 제공해야 합니다. 만약 요구사항이 없다면 빈 배열 `[]`을 반환하십시오.
    불필요한 서론, 사업 배경, 계약 조건, 제안 지침 등은 요구사항으로 추출하지 마십시오. PDF의 '상세 요구사항' 목록에 있는 구조화된 항목들 위주로 추출해주십시오.
    """

    for i, chunk_doc in enumerate(initial_chunks):
        chunk_text = chunk_doc.page_content
        print(f"--- LLM 처리 중 (요구사항 상세 추출): 청크 {i+1}/{len(initial_chunks)} (길이: {len(chunk_text)}자) ---")

        page_numbers_in_chunk = sorted(list(set(
            int(p) for p in re.findall(r'---PAGE_START_(\d+)---', chunk_text)
        )))

        user_prompt = f"""
        다음은 RFP 문서의 일부입니다. 이 부분에서 모든 시스템 요구사항을 개발자 표준에 맞춰 JSON 형식으로 추출해 주세요.
        주어진 텍스트 내의 "요구사항 고유번호", "요구사항 명칭", "요구사항 분류", "정의", "상세설명/세부내용" 등의 명시적 필드를 최대한 활용하여 JSON 객체를 구성해주십시오.

        --- 텍스트 ---
        {chunk_text}
        """

        try:
            response = client_instance.chat.completions.create(
                model=llm_model,
                response_format={"type": "json_object"},
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.0
            )

            llm_response_content = response.choices[0].message.content
            try:
                extracted_data_outer = json.loads(llm_response_content)
                extracted_data_list = []
                if isinstance(extracted_data_outer, list):
                    extracted_data_list = extracted_data_outer
                elif isinstance(extracted_data_outer, dict):
                    # 딕셔너리 값 중에 리스트를 찾아 첫 번째 것을 사용 (일반적인 LLM 응답 패턴)
                    for key_in_dict in extracted_data_outer:
                        if isinstance(extracted_data_outer[key_in_dict], list):
                            extracted_data_list = extracted_data_outer[key_in_dict]
                            break
                    if not extracted_data_list: # 그래도 못찾으면 경고
                        print(f"경고: LLM이 JSON 객체를 반환했으나, 그 안에 예상된 요구사항 리스트를 찾지 못했습니다. 객체 키: {list(extracted_data_outer.keys())}")
                else:
                    print(f"경고: LLM으로부터 예상치 못한 JSON 형식 응답 (리스트 또는 객체 내 리스트가 아님). {llm_response_content[:100]}...")
                    continue
            except json.JSONDecodeError as e_inner:
                print(f"LLM 응답 내용 JSON 파싱 오류 (내부 시도): {e_inner}. 응답: {llm_response_content[:500]}...")
                continue

            for req_idx, req in enumerate(extracted_data_list):
                # 'id'가 없거나 비어있으면 생성, 또는 원본 ID 우선 사용
                original_id = req.get('id') # LLM이 원본 ID (ECR-001 등)를 id 필드에 넣어줬길 기대
                if not original_id:
                    req_type_prefix = "REQ"
                    req_type = req.get('type', '기타').strip()
                    if "기능" in req_type: req_type_prefix = "FUNC"
                    elif "비기능" in req_type: req_type_prefix = "NFR"
                    elif "성능" in req_type: req_type_prefix = "PERF"
                    elif "보안" in req_type: req_type_prefix = "SEC"
                    elif "데이터" in req_type: req_type_prefix = "DATA"
                    elif "제약" in req_type: req_type_prefix = "CONST"
                    elif "장비" in req_type: req_type_prefix = "ECR"
                    elif "컨설팅" in req_type: req_type_prefix = "CSR"
                    elif "테스트" in req_type: req_type_prefix = "TER"
                    elif "품질" in req_type: req_type_prefix = "QUR"
                    elif "관리" in req_type: req_type_prefix = "PMR"
                    elif "지원" in req_type: req_type_prefix = "PSR"
                    req_id_counter += 1
                    req['id'] = f"{req_type_prefix}-{req_id_counter:03d}"

                if 'source_pages' not in req or not req['source_pages']:
                    req['source_pages'] = page_numbers_in_chunk if page_numbers_in_chunk else []

                if 'raw_text_snippet' not in req or not req['raw_text_snippet']:
                    req['raw_text_snippet'] = f"청크 {i+1}에서 추출됨. 원본 청크 일부: {chunk_text[:200]}..." # 개선 필요

                req['type'] = req.get('type', '기타')
                req['description'] = req.get('description', '설명 없음')
                req['acceptance_criteria'] = req.get('acceptance_criteria', '해당하는 경우 명시')
                req['priority'] = req.get('priority', '필수')
                req['responsible_module'] = req.get('responsible_module', '미정')
                final_semantic_chunks.append(req)

        except json.JSONDecodeError as e:
            print(f"LLM 응답 JSON 파싱 오류: {e}. 응답: {llm_response_content[:500]}...")
        except Exception as e:
            print(f"LLM API 호출 또는 처리 중 오류 발생: {e}")

    return final_semantic_chunks


# --- LLM으로 목차 파싱하는 함수 ---
def parse_toc_with_llm(toc_raw_text, client_instance, llm_model="gpt-4o"):
    """
    LLM을 사용하여 목차 원문 텍스트를 파싱하여 구조화된 목차 항목을 추출합니다.
    """
    system_prompt = """
    당신은 PDF 문서에서 추출된 목차(Table of Contents)의 원시 텍스트를 분석하는 전문가입니다.
    주어진 텍스트를 파싱하여 각 목차 항목의 제목, 페이지 번호, 그리고 해당 항목이 '요구사항' 관련 내용을 담고 있을 가능성을 분석하여 JSON 형태로 반환해야 합니다.
    JSON의 최상위 레벨은 "toc_entries"라는 키를 가진 객체여야 하고, 그 키의 값은 목차 항목 객체들의 리스트여야 합니다.
    """
    user_prompt = f"""
    다음은 PDF에서 추출한 목차로 추정되는 텍스트입니다:

    --- 목차 원문 텍스트 시작 ---
    {toc_raw_text}
    --- 목차 원문 텍스트 끝 ---

    이 텍스트를 분석하여 JSON 객체를 반환해주세요. 이 객체는 "toc_entries"라는 키를 가져야 하며,
    이 키의 값은 각 목차 항목을 나타내는 객체들의 리스트여야 합니다.
    각 목차 항목 객체는 다음 키를 가져야 합니다:
    - "title": (문자열) 목차 항목의 전체 제목. 제목 앞의 번호(예: "1.", "II.", "가.")도 포함해주세요.
    - "page": (정수) 해당 항목의 시작 페이지 번호.
    - "is_requirement_related": (불리언) 제목이나 내용을 볼 때, 해당 항목이 '요구사항', '과업 범위', '제안 요청 상세', '기능 명세', '기술 요건' 등과 관련된 내용을 다룰 가능성이 높으면 true, 그렇지 않으면 false로 설정해주세요.

    예시 JSON 출력 형식:
    {{
      "toc_entries": [
        {{
          "title": "1. 사업 개요",
          "page": 5,
          "is_requirement_related": false
        }},
        {{
          "title": "III. 제안요청 내용",
          "page": 6,
          "is_requirement_related": true
        }},
        {{
          "title": "3. 상세 요구사항",
          "page": 11,
          "is_requirement_related": true
        }},
        {{
          "title": "* 보안 요구사항 별표",
          "page": 63,
          "is_requirement_related": true
        }}
      ]
    }}

    만약 주어진 텍스트가 유효한 목차로 보이지 않거나 항목을 전혀 파싱할 수 없다면,
    "toc_entries" 키의 값으로 빈 리스트 `[]`를 포함하는 JSON 객체를 반환해주세요. (예: {{"toc_entries": []}})
    """
    llm_response_content = "" # 오류 발생 시 로깅을 위해 미리 선언
    try:
        response = client_instance.chat.completions.create(
            model=llm_model,
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.0
        )
        llm_response_content = response.choices[0].message.content
        # print(f"DEBUG: LLM RAW RESPONSE FOR TOC: {llm_response_content}")

        parsed_data = json.loads(llm_response_content)

        extracted_list = []
        if isinstance(parsed_data, dict) and "toc_entries" in parsed_data and isinstance(parsed_data["toc_entries"], list):
            extracted_list = parsed_data["toc_entries"]
        else:
            print(f"LLM 응답이 예상된 'toc_entries' 리스트를 포함하는 객체 형식이 아닙니다. 응답: {llm_response_content[:200]}")
            return [] # 빈 리스트 반환

        valid_entries = []
        for entry in extracted_list:
            if isinstance(entry, dict) and 'title' in entry and 'page' in entry:
                try:
                    entry['page'] = int(entry['page'])
                    entry['is_requirement_related'] = bool(entry.get('is_requirement_related', False)) # 명확히 불리언으로
                    valid_entries.append(entry)
                except ValueError:
                    print(f"경고: 페이지 번호 '{entry.get('page')}'를 정수로 변환할 수 없습니다. 항목 건너뜀: {entry.get('title')}")
                    continue
            else:
                print(f"경고: 필수 키(title, page)가 누락된 항목입니다. 건너뜀: {entry}")

        return valid_entries

    except json.JSONDecodeError as e:
        print(f"LLM 목차 파싱 응답 JSON 파싱 오류: {e}. 응답 미리보기: {llm_response_content[:500]}...")
        return []
    except Exception as e:
        print(f"LLM API 호출 또는 처리 중 오류 발생 (목차 파싱): {e}")
        return []

# --- 새로운 헬퍼 함수들 ---
def extract_text_for_pages(full_text_with_pages, start_page_num, end_page_num):
    """
    전체 텍스트에서 특정 페이지 범위의 텍스트만 추출합니다.
    페이지 마커 '---PAGE_START_N---'를 사용합니다.
    """

    # 페이지 범위 유효성 검사 (end_page_num이 start_page_num보다 작을 수 없음)
    if end_page_num < start_page_num:
        # print(f"경고: 끝 페이지({end_page_num})가 시작 페이지({start_page_num})보다 작습니다. 빈 텍스트를 반환합니다.")
        return ""

    # 패턴 구성: 시작 페이지부터 (끝 페이지 + 1) 직전까지 모든 내용을 포함
    # re.escape를 사용하여 페이지 마커의 특수 문자를 이스케이프할 필요는 여기서는 없음
    # DOTALL 플래그를 사용하여 \n도 .에 매치되도록 함

    # 시작점 찾기
    start_marker = f"---PAGE_START_{start_page_num}---"
    start_match = re.search(re.escape(start_marker), full_text_with_pages) # 마커 자체를 찾아야 함

    if not start_match:
        # print(f"경고: 시작 마커 '{start_marker}'를 찾을 수 없습니다.")
        return ""

    text_from_start_page = full_text_with_pages[start_match.start():]

    # 끝점 찾기 (다음 페이지 마커 ---PAGE_START_{end_page_num + 1}---)
    # end_page_num이 문서의 마지막 페이지일 수도 있으므로, end_page_num + 1 마커가 없을 수 있음
    end_marker_exclusive = f"---PAGE_START_{end_page_num + 1}---"
    end_match = re.search(re.escape(end_marker_exclusive), text_from_start_page)

    if end_match:
        return text_from_start_page[:end_match.start()]
    else:
        # end_page_num + 1 마커를 찾지 못하면, start_page_num부터 문서 끝까지 반환 (또는 start_page_num ~ end_page_num의 마지막 내용까지)
        # 이 경우는 end_page_num이 마지막 페이지이거나, 그 이후 페이지 마커가 없는 경우.
        # 좀 더 정확하게 하려면, end_page_num까지의 모든 내용을 가져와야 함.
        # 그러나 페이지 마커 구조상, end_page_num의 내용은 end_page_num+1 마커 전까지임.
        # 따라서 end_match가 없으면 text_from_start_page 전체가 해당 범위임.
        return text_from_start_page


def get_target_sections_from_llm_parsed_toc(parsed_toc_entries, total_pages):
    """
    LLM으로 파싱된 목차에서 '상세 요구사항' 및 '보안 요구사항' 관련 섹션 정보를 추출합니다.
    """
    target_sections = []

    # 페이지 번호 기준으로 목차 정렬 (LLM이 순서대로 안 줄 수도 있으므로)
    sorted_toc = sorted(parsed_toc_entries, key=lambda x: x.get('page', 0))

    # 주요 키워드 - 사용자가 제공한 PDF 목차 기반
    # "III. 제안요청 내용" 안에 "3. 상세 요구사항"이 있으므로, "상세 요구사항"을 찾는 것이 더 정확함.
    keywords_to_find = {
        "상세 요구사항": {"min_pages": 5}, # 최소한 이정도는 될 것이라는 기대
        "보안 요구사항 별표": {"min_pages": 1} # 별표는 짧을 수도 있음
        # 필요시 다른 주요 섹션 키워드 추가 가능
    }

    for i, entry in enumerate(sorted_toc):
        entry_title = entry.get('title', '').strip()
        entry_page = entry.get('page', 0)

        for keyword, props in keywords_to_find.items():
            if keyword in entry_title:
                start_page = entry_page
                end_page = total_pages # 기본값: 문서 끝까지

                # 다음 목차 항목의 시작 페이지 - 1을 현재 섹션의 끝 페이지로 설정
                # 단, 다음 항목이 현재 항목과 같은 페이지에서 시작하면 안됨 (하위 항목일 수 있으므로)
                # 좀 더 정교한 로직: 다음 '주요' 항목을 찾아야 함.
                # 여기서는 일단 다음 항목의 시작 페이지를 사용.
                # 만약 다음 항목이 현재 항목의 하위 항목처럼 보이면 (예: "3. 상세 요구사항" 다음 "3.1 기능 요구사항")
                # 그 하위 항목의 범위를 포함하도록 확장하거나, 아니면 정말 다음 *다른* 주요 섹션까지 봐야 함.
                # 현재 LLM 프롬프트는 is_requirement_related로 판단하므로, 이를 우선적으로 신뢰.

                # 다음 'is_requirement_related=False'인 섹션 또는 다음 주요 섹션(로마숫자/대문자 알파벳 등)을 찾아 end_page 설정
                # 또는 단순히 다음 목차 항목의 시작 페이지 - 1 로 설정
                next_major_section_page = total_pages + 1 # 충분히 큰 값
                for j in range(i + 1, len(sorted_toc)):
                    next_entry = sorted_toc[j]
                    next_entry_page = next_entry.get('page', 0)
                    if next_entry_page > start_page : # 현재 섹션 이후의 페이지만 고려
                        # 여기서 '주요 섹션'을 판단하는 기준이 필요함 (예: 로마 숫자, 대문자 번호 등)
                        # 또는 is_requirement_related=False 인 첫번째 섹션
                        # 또는 단순히 다음 목차 항목
                        next_major_section_page = next_entry_page
                        break

                end_page = min(total_pages, next_major_section_page -1)

                # 시작 페이지와 끝 페이지 유효성 확보
                if end_page < start_page:
                    end_page = start_page

                # 페이지 수가 너무 적으면 해당 섹션 전체를 포함 (예: 11페이지로 나왔는데 실제로는 62까지일 수 있음)
                # 이 부분은 heuristic이므로 주의. 더 좋은 방법은 LLM에게 섹션의 끝을 명확히 묻는 것.
                # 여기서는 min_pages 이상은 되어야 유의미하다고 가정.
                # if (end_page - start_page + 1) < props.get("min_pages", 1) and (start_page + props.get("min_pages", 1) -1) <= total_pages :
                #     pass # end_page = start_page + props.get("min_pages", 1) -1 # 최소 페이지 강제는 위험할 수 있음

                target_sections.append({
                    'title': entry_title,
                    'start_page': start_page,
                    'end_page': end_page
                })
                print(f"LLM 파싱 목차 기반: '{entry_title}' 섹션 (페이지 {start_page}-{end_page}) 식별")
                # 찾은 키워드는 중복 추가 방지 (더 구체적인 항목이 먼저 찾아지도록 keywords_to_find 순서 중요)
                break

    # 만약 아무것도 못찾았지만, is_requirement_related=True 인 항목들이 있다면 그것들을 사용
    if not target_sections:
        print("키워드 기반 섹션 식별 실패. 'is_requirement_related=True' 플래그로 섹션 식별 시도...")
        for i, entry in enumerate(sorted_toc):
            if entry.get('is_requirement_related'):
                start_page = entry.get('page',0)
                end_page = total_pages
                # 위와 동일한 로직으로 end_page 계산
                next_major_section_page = total_pages + 1
                for j in range(i + 1, len(sorted_toc)):
                    next_entry = sorted_toc[j]
                    next_entry_page = next_entry.get('page', 0)
                    if next_entry_page > start_page:
                        if not next_entry.get('is_requirement_related', False): # 다음 주요 섹션이 요구사항 관련이 아니면
                            next_major_section_page = next_entry_page
                            break
                        # 또는 다음 항목이 현재 항목보다 상위 레벨이면 (예: "3.1" 다음 "4.")
                        # 이 부분은 제목의 번호 체계를 분석해야 해서 복잡함. LLM의 is_requirement_related를 신뢰.

                end_page = min(total_pages, next_major_section_page - 1)
                if end_page < start_page: end_page = start_page

                target_sections.append({
                    'title': entry.get('title', '요구사항 관련 섹션'),
                    'start_page': start_page,
                    'end_page': end_page
                })
                print(f"LLM 'is_requirement_related' 플래그 기반: '{entry.get('title')}' 섹션 (페이지 {start_page}-{end_page}) 식별")

    # 중복 제거 및 병합 (예: "III. 제안요청 내용"과 그 하위 "3. 상세 요구사항"이 모두 선택된 경우)
    # 여기서는 단순화를 위해 중복된 페이지 범위가 있다면 가장 포괄적인 것을 선택하거나,
    # 또는 가장 구체적인 "상세 요구사항"을 우선. 지금은 일단 나온대로 반환.
    # 더 나은 방법은, '상세 요구사항'이 나왔으면 그게 더 우선순위가 높다고 처리.

    if not target_sections:
        print("경고: LLM 파싱 목차에서 주요 요구사항 섹션을 찾지 못했습니다. 전체 문서를 대상으로 할 수 있습니다.")
        return [{'title': '전체 문서 (목차 분석 실패)', 'start_page': 1, 'end_page': total_pages}]

    return target_sections


def get_toc_raw_text_from_full_text(full_text_with_pages, toc_page_numbers=[2,3]):
    """
    전체 텍스트에서 지정된 목차 페이지들의 텍스트만 추출합니다.
    """
    toc_texts = []
    for page_num in toc_page_numbers:
        # 페이지 마커를 찾아서 해당 페이지의 텍스트를 추출
        # (end_page_num + 1) 마커를 찾아서 그 전까지의 내용을 가져옴
        page_content = extract_text_for_pages(full_text_with_pages, page_num, page_num)
        if page_content:
            # 페이지 마커 제거 (이미 extract_text_for_pages가 마커 다음부터 가져오지만, 혹시 몰라서)
            # 실제로는 페이지 마커 이후의 텍스트만 필요.
            # `extract_text_for_pages`는 마커를 포함해서 반환할 수 있으므로, 마커 이후 내용만 사용.
            marker = f"---PAGE_START_{page_num}---\n"
            if page_content.startswith(marker):
                toc_texts.append(page_content[len(marker):])
            else: # 마커가 없는 경우 (예상치 않음)
                toc_texts.append(page_content)
        else:
            print(f"경고: 목차 페이지로 지정된 {page_num} 페이지에서 텍스트를 찾을 수 없습니다.")

    if not toc_texts:
        return None
    return "\n".join(toc_texts)

In [47]:

def decompose_requirement_with_llm(
    parent_requirement: Dict[str, Any],
    sub_requirement_id_prefix_to_use: str, # 하위 요구사항 ID에 사용할 접두사
    sub_req_id_counters: Dict[str, int],    # 접두사별 ID 순번 카운터 (이 함수 호출 간에 상태 유지)
    client_instance: OpenAI,
    llm_model: str
) -> List[Dict[str, Any]]:
    """
    상위 요구사항을 LLM을 사용하여 세분화된 하위 요구사항 리스트로 변환합니다.
    하위 요구사항 ID는 sub_req_id_counters를 참조하여 전체 실행에서 고유하게 생성됩니다.
    세분화 수준을 '개발에 적당한 레벨'로 조정합니다.
    """
    parent_id = parent_requirement.get("id", "UNKNOWN_PARENT")
    parent_description = parent_requirement.get("description", "")
    parent_acceptance_criteria = parent_requirement.get("acceptance_criteria", "")
    parent_type = parent_requirement.get("type", "기능적")
    parent_module = parent_requirement.get("responsible_module", "미정")
    parent_source_pages = parent_requirement.get("source_pages", []) # 이 필드가 실제 JSON에 있는지 확인 필요


    example_sub_tasks_description_style = """
    예를 들어, "통계 관리 기능"을 세분화 한다면 다음과 같은 주요 기능 단위가 나올 수 있습니다 (너무 상세한 개별 항목보다는 기능 그룹에 집중):
    - 주요 통계 지표 조회 기능 (교육 과정별, 연도별, 지역별, 기간별 등 주요 필터 포함)
    - 웹사이트 방문자 행동 분석 통계 기능 (일자별/단말기별/접속경로별 방문자 수 및 주요 행동 패턴)
    - 학습 현황 대시보드 제공 기능 (핵심 지표 시각화 및 요약)
    - 통계 데이터 관리 기능 (데이터 수집, 검증, 로그 관리 등)
    - 통계 리포팅 및 다운로드 기능 (주요 통계 화면의 보고서 생성 및 데이터 추출)
    이처럼 각 하위 요구사항은 개발팀이 하나의 의미 있는 기능 단위로 인식하고 작업을 계획할 수 있는 수준이어야 합니다.
    지나치게 많은 수(예: 20개 이상)의 매우 작은 단위로 분해하기보다는, 5~10개 내외의 핵심 하위 기능으로 그룹화하는 것이 좋습니다.
    """

    system_prompt = f"""
    당신은 시스템 요구사항 분석 및 설계 전문가입니다. 주어진 상위 요구사항을 **실제로 개발팀이 작업을 분담하고 진행할 수 있는 적절한 크기의 주요 하위 기능들로 세분화**하는 임무를 받았습니다.
    각 하위 요구사항은 그 자체로 의미 있는 기능 단위를 나타내야 하며, 독립적으로 개발 및 테스트가 가능해야 합니다. **너무 과도하게 많은 수의 지나치게 세세한 항목으로 나누는 것은 지양합니다.**

    다음은 세분화할 상위 요구사항의 정보입니다:
    - 상위 ID: {parent_id}
    - 상위 요구사항 명: {parent_description}
    - 상위 요구사항 인수 조건: {parent_acceptance_criteria}
    - (참고) 상위 요구사항 유형: {parent_type}, 담당 모듈: {parent_module}, 출처 페이지: {parent_source_pages}

    아래는 "{parent_description}"와 유사한 주제에 대한 세분화 예시입니다. **이 예시에서 제시된 세분화의 '수준'과 '단위의 크기'를 참고**하여 하위 요구사항들을 도출하십시오:
    {example_sub_tasks_description_style}

    각 하위 요구사항에 대해 다음 JSON 형식을 사용하여 상세 정보를 제공해주십시오.
    `id` 필드는 "TBD"로 설정해주십시오. (외부에서 `{sub_requirement_id_prefix_to_use}-XXX` 형식으로 부여됨)
    다른 필드들은 상위 요구사항 정보와 하위 요구사항의 내용을 바탕으로 적절히 추론하거나 상속하여 채워주십시오.
    `description` 필드는 하위 요구사항의 명칭을, `detailed_description` 필드는 그에 대한 상세 설명을 담도록 합니다. (만약 명칭과 설명이 거의 같다면 `detailed_description`은 `description`과 동일하게 작성해도 무방합니다.)

    반환 형식은 반드시 하위 요구사항 객체들을 담고 있는 JSON 배열 `[]` 이어야 합니다. 만약 최상위가 배열이 아닌 JSON 객체여야 한다면, **반드시 `sub_requirements` 라는 키의 값으로 하위 요구사항 배열을 제공**하십시오.
    주어진 상위 요구사항이 이미 충분히 구체적이어서 더 이상 의미 있는 단위로 세분화하기 어렵다고 판단되면, 빈 배열 `[]`을 반환하십시오.
    """

    user_prompt = f"""
    상위 요구사항 (ID: {parent_id}, 명칭: "{parent_description}")을 위 지침과 예시에 따라, **너무 많지 않은 수의 적절한 개발 단위로** 세분화된 하위 기능 요구사항 리스트로 만들어 주십시오. (이상적으로 5~10개 내외의 주요 하위 기능)
    각 하위 요구사항의 JSON 객체는 다음 필드를 포함해야 합니다:
    - id: (문자열, "TBD"로 설정)
    - type: (문자열, 상위 요구사항 유형 '{parent_type}'을 따르거나 "기능적"으로 설정)
    - description: (문자열, 세분화된 요구사항의 명칭, 예시 스타일 참고)
    - acceptance_criteria: (문자열, 세분화된 요구사항의 인수 조건)
    - responsible_module: (문자열, 상위 담당 모듈 '{parent_module}' 상속 또는 더 구체적인 모듈)
    - parent_id: (문자열, "{parent_id}")
    - source_pages: (정수 리스트, 상위 출처 페이지 {parent_source_pages} 상속)
    """

    llm_response_str = "" # 오류 로깅을 위해 초기화
    try:
        completion = client_instance.chat.completions.create(
            model=llm_model,
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.2, 
        )
        llm_response_str = completion.choices[0].message.content
        logging.debug(f"LLM Raw Response for parent '{parent_id}':\n{llm_response_str[:500]}...")

        raw_llm_output_data = json.loads(llm_response_str)
        
        extracted_sub_reqs_from_llm = []
        if isinstance(raw_llm_output_data, dict) and "sub_requirements" in raw_llm_output_data and isinstance(raw_llm_output_data["sub_requirements"], list):
            extracted_sub_reqs_from_llm = raw_llm_output_data["sub_requirements"]
        elif isinstance(raw_llm_output_data, list):
             extracted_sub_reqs_from_llm = raw_llm_output_data
             logging.warning(f"LLM이 '{parent_id}'에 대해 직접 리스트를 반환했습니다 (예상: 객체 내 'sub_requirements' 키).")
        else:
            logging.warning(f"LLM이 '{parent_id}'에 대해 예상된 'sub_requirements' 리스트 형식을 반환하지 않음. 응답 타입: {type(raw_llm_output_data)}. 응답 앞부분: {str(raw_llm_output_data)[:200]}")
            return []

        final_processed_sub_requirements = []
        for sub_req_data_from_llm in extracted_sub_reqs_from_llm:
            if not isinstance(sub_req_data_from_llm, dict) or not sub_req_data_from_llm.get("description"):
                logging.warning(f"'{parent_id}'의 하위 요구사항 중 유효하지 않은 항목 발견(description 누락 등): {sub_req_data_from_llm}")
                continue

            current_count = sub_req_id_counters.get(sub_requirement_id_prefix_to_use, 0) + 1
            sub_req_id_counters[sub_requirement_id_prefix_to_use] = current_count
            
            processed_sub_req = {
                "id": f"{sub_requirement_id_prefix_to_use}-{current_count:03d}",
                "type": sub_req_data_from_llm.get("type", parent_type),
                "description": sub_req_data_from_llm.get("description"),
                "acceptance_criteria": sub_req_data_from_llm.get("acceptance_criteria", "세부 인수 조건 정의 필요"),
                "responsible_module": sub_req_data_from_llm.get("responsible_module", parent_module),
                "parent_id": parent_id,
                "source_pages": parent_source_pages 
            }
            final_processed_sub_requirements.append(processed_sub_req)
            
        return final_processed_sub_requirements

    except json.JSONDecodeError as e:
        logging.error(f"'{parent_id}' 세분화 중 LLM 응답 JSON 파싱 실패: {e}. 응답 내용(앞 500자): {llm_response_str[:500]}")
        return []
    except Exception as e:
        logging.error(f"'{parent_id}' 세분화 중 예기치 않은 오류 발생: {e}", exc_info=True)
        return []
    


def batch_decompose_requirements_from_file(
    input_json_path: str,
    output_json_path: str,
    default_sub_req_id_prefix: str,
    client_instance: OpenAI,
    llm_model: str
):
    """
    입력 JSON 파일에서 상위 요구사항 리스트를 읽어 각각을 세분화하고,
    모든 생성된 하위 요구사항을 취합하여 출력 JSON 파일로 저장합니다.
    """
    try:
        with open(input_json_path, 'r', encoding='utf-8') as f:
            all_parent_requirements = json.load(f)
    except FileNotFoundError:
        logging.error(f"입력 JSON 파일 '{input_json_path}'를 찾을 수 없습니다.")
        return
    except json.JSONDecodeError as e:
        logging.error(f"입력 JSON 파일 '{input_json_path}' 파싱 오류: {e}")
        return
    except Exception as e:
        logging.error(f"입력 JSON 파일 '{input_json_path}' 로드 중 오류: {e}", exc_info=True)
        return

    if not isinstance(all_parent_requirements, list):
        logging.error(f"입력 JSON 데이터가 리스트 형식이 아닙니다 (타입: {type(all_parent_requirements)}).")
        return

    all_generated_sub_requirements: List[Dict] = []
    sub_req_id_counters_map: Dict[str, int] = {} 

    for parent_req_index, parent_req_data in enumerate(all_parent_requirements):
        if not isinstance(parent_req_data, dict):
            logging.warning(f"입력 파일의 {parent_req_index+1}번째 항목이 딕셔너리가 아닙니다. 건너<0xEB><0x9A><0x84>니다: {parent_req_data}")
            continue
        
        parent_id_for_log = parent_req_data.get("id", f"UNKNOWN_PARENT_{parent_req_index+1}")
        logging.info(f"\n--- {parent_req_index+1}/{len(all_parent_requirements)}번째 상위 요구사항 '{parent_id_for_log}' 세분화 시작 ---")
        
        
        decomposed_list = decompose_requirement_with_llm(
            parent_requirement=parent_req_data,
            sub_requirement_id_prefix_to_use=default_sub_req_id_prefix,
            sub_req_id_counters=sub_req_id_counters_map,
            client_instance=client_instance,
            llm_model=llm_model
        )
        
        if decomposed_list:
            all_generated_sub_requirements.extend(decomposed_list)
        else:
            logging.warning(f"상위 요구사항 '{parent_id_for_log}'에 대해 하위 요구사항이 생성되지 않았습니다. 원본 요구사항을 유지하거나 다른 처리를 할 수 있습니다.")

    logging.info(f"\n--- 총 {len(all_generated_sub_requirements)}개의 하위 요구사항 생성 완료 ---")

    try:
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(all_generated_sub_requirements, f, ensure_ascii=False, indent=4)
        logging.info(f"모든 세분화된 요구사항이 '{output_json_path}' 파일로 성공적으로 저장되었습니다.")
    except IOError as e:
        logging.error(f"출력 JSON 파일 '{output_json_path}' 쓰기 중 I/O 오류: {e}")
    except Exception as e:
        logging.error(f"출력 JSON 파일 저장 중 예기치 않은 오류: {e}", exc_info=True)

In [ ]:

# --- 메인 실행 블록 ---
if __name__ == "__main__":
    pdf_file_path = "docs/제주은행_RFP.pdf" # 실제 파일 경로
    if not os.path.exists(pdf_file_path):
        print(f"오류: PDF 파일을 찾을 수 없습니다 - {pdf_file_path}")
        exit()

    print("1. PDF 전체 텍스트 추출 중...")
    full_document_text, total_pages = extract_text_with_page_info(pdf_file_path)
    if not full_document_text or total_pages == 0:
        print(f"오류: PDF에서 텍스트를 추출하지 못했거나 페이지 수가 0입니다 ({pdf_file_path}).")
        exit()
    print(f"   PDF 전체 텍스트 추출 완료. 총 {total_pages} 페이지, 전체 텍스트 길이: {len(full_document_text)}자.")

    print("\n2. 목차(ToC) 원문 텍스트 추출 중 (지정 페이지: 2, 3)...")
    # 사용자 PDF 분석 결과 PAGE 2, 3에 목차가 있음
    toc_raw_text = get_toc_raw_text_from_full_text(full_document_text, toc_page_numbers=[2, 3])

    target_sections_for_extraction = []
    if toc_raw_text:
        print(f"   목차 원문 텍스트 추출 완료 (길이: {len(toc_raw_text)}자).")
        print("\n3. LLM을 사용하여 목차(ToC) 파싱 중...")
        parsed_toc_entries = parse_toc_with_llm(toc_raw_text, client, llm_model=LLM_MODEL_FOR_PARSING)

        if parsed_toc_entries:
            print(f"   LLM 목차 파싱 완료. {len(parsed_toc_entries)}개의 목차 항목 식별.")
            print("\n4. 파싱된 목차에서 주요 요구사항 섹션 범위 식별 중...")
            target_sections_for_extraction = get_target_sections_from_llm_parsed_toc(parsed_toc_entries, total_pages)
        else:
            print("   LLM 목차 파싱 실패. 전체 문서를 대상으로 분석합니다.")
            target_sections_for_extraction = [{'title': '전체 문서 (LLM 목차 파싱 실패)', 'start_page': 1, 'end_page': total_pages}]
    else:
        print("   목차 원문 텍스트 추출 실패. 전체 문서를 대상으로 분석합니다.")
        target_sections_for_extraction = [{'title': '전체 문서 (목차 원문 추출 실패)', 'start_page': 1, 'end_page': total_pages}]

    if not target_sections_for_extraction: # 만약을 위해 한번 더 체크
        print("오류: 분석할 대상 섹션을 정의하지 못했습니다. 프로그램을 종료합니다.")
        exit()

    print("\n5. 식별된 주요 섹션으로부터 텍스트 결합 중...")
    all_requirements_related_text_parts = []
    for section_info in target_sections_for_extraction:
        print(f"   '{section_info['title']}' 섹션 (페이지 {section_info['start_page']}-{section_info['end_page']}) 텍스트 추출 시도...")
        section_text = extract_text_for_pages(full_document_text, section_info['start_page'], section_info['end_page'])
        if section_text:
            all_requirements_related_text_parts.append(section_text)
            print(f"       '{section_info['title']}' 섹션 텍스트 추출 완료 (길이: {len(section_text)}자).")
        else:
            print(f"       경고: '{section_info['title']}' 섹션 (페이지 {section_info['start_page']}-{section_info['end_page']})에서 텍스트를 추출하지 못했습니다.")

    if not all_requirements_related_text_parts:
        print("오류: 주요 요구사항 섹션에서 텍스트를 전혀 추출하지 못했습니다. 프로그램을 종료합니다.")
        exit()

    combined_requirements_text = "\n\n".join(all_requirements_related_text_parts) # 섹션 간 구분을 위해 두 줄바꿈
    print(f"   주요 섹션 텍스트 결합 완료. 총 길이: {len(combined_requirements_text)}자.")

    print("\n6. 결합된 텍스트 초기 분할 중...")
    # chunk_size는 LLM의 context window와 한글 토큰 소모를 고려하여 설정
    initial_chunks = initial_text_split(combined_requirements_text, chunk_size=3500, chunk_overlap=350)
    print(f"   총 {len(initial_chunks)}개의 초기 청크가 생성되었습니다.")

    if not initial_chunks:
        print("오류: 텍스트 분할 결과 청크가 없습니다. 프로그램을 종료합니다.")
        exit()

    print("\n7. LLM을 이용한 개발자 표준 요구사항 상세 추출 시작...")
    extracted_requirements = llm_based_semantic_chunking_for_dev_reqs(initial_chunks, client, llm_model=LLM_MODEL_FOR_PARSING)

    print("\n8. 후처리: ID 중복 제거 및 최종 정렬 (옵션)...")
    # (기존 스크립트의 후처리 로직 사용 또는 필요시 개선)
    unique_requirements_by_id = {}
    processed_requirements = []
    if extracted_requirements:
        for req in extracted_requirements:
            req_id = req.get('id', str(uuid.uuid4())) # ID가 없다면 UUID로 임시 ID
            if req_id not in unique_requirements_by_id:
                unique_requirements_by_id[req_id] = req
            else:
                # ID가 중복될 경우, 설명을 합치거나 페이지 정보를 합치는 등의 로직 추가 가능
                # 여기서는 간단히 첫 번째 것만 유지하거나, 필요에 따라 업데이트
                # 예: 페이지 정보 병합
                existing_req = unique_requirements_by_id[req_id]
                new_pages = req.get('source_pages', [])
                if new_pages:
                    existing_req_pages = existing_req.get('source_pages', [])
                    existing_req['source_pages'] = sorted(list(set(existing_req_pages + new_pages)))
                print(f"중복 ID '{req_id}' 감지. 정보 업데이트 시도.")
        processed_requirements = list(unique_requirements_by_id.values())
    else:
        print("   추출된 요구사항이 없습니다.")

    # 최종 결과 출력 또는 저장
    print(f"\n--- 최종 추출된 고유 요구사항 수: {len(processed_requirements)}개 ---")
    if processed_requirements:
        print("\n--- 추출된 요구사항 미리보기 (상위 3개) ---")
        for i, req_item in enumerate(processed_requirements[:3]): # 'req' 변수명 충돌 피하기 위해 'req_item'으로 변경
            print(json.dumps(req_item, ensure_ascii=False, indent=2))
            print("-" * 30)

        output_filename = "extracted_requirements_final.json"
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(processed_requirements, f, ensure_ascii=False, indent=4)
        print(f"\n추출된 요구사항이 '{output_filename}' 파일에 저장되었습니다.")
    else:
        print("최종적으로 추출된 요구사항이 없습니다.")
    

# Langgraph

In [48]:
from openai import OpenAI # 실제 실행 시 필요
import concurrent.futures
import os # For API Key environment variable (recommended)
from typing import TypedDict, Dict, Any, List, Optional

# 작업 1: 요구사항 분류
def generate_classification_prompt_text(description, detailed_description, module):
    return f"""
당신은 차세대 정보시스템 구축 프로젝트에서 요구사항을 분석하고, 아래 기준에 따라 대분류, 중분류, 소분류를 분류하는 전문가입니다.

다음 요구사항 설명을 읽고 각 분류 항목에 맞게 분류해 주세요:

[요구사항 설명]
{description}

[상세 설명]
{detailed_description}

[담당 모듈]
{module}

[분류 기준]
1. **대분류**: 차세대 정보시스템 업무 수준
   예시: 수신, 여신, 부대/대행, 통합고객 등

2. **중분류**: 단위업무 시스템 수준
   예시: 예금, 신탁, 상담신청, 심사승인 등

3. **소분류**: 단위업무 시스템 하위 수준 (업무 프로세스 3~4레벨)
   ※ 소분류는 3레벨 분류가 어려운 경우 선택적으로 작성해도 무방함

아래 형식으로 정확히 출력하세요 (불필요한 설명 없이):

대분류: <텍스트>
중분류: <텍스트>
소분류: <텍스트 또는 '해당 없음'>

※ 유의사항:
- 반드시 대분류 → 중분류 → 소분류 순으로 작성
- 각 분류명은 명확하고 직관적인 한국어 명사형 표현을 사용할 것
- 기존 분류 체계가 없으므로, 의미적으로 유사한 요구사항끼리 논리적으로 묶어서 계층화할 것
- 불필요한 설명 없이 위 형식만 출력
"""

def classify_requirement_logic(description: str, detailed_description: str, module: str) -> Dict[str, str]:
    prompt = generate_classification_prompt_text(description, detailed_description, module)
    try:
        response = client.chat.completions.create(
            model="gpt-4", # 또는 gpt-4-turbo, gpt-3.5-turbo 등 사용 가능한 모델
            messages=[
                {"role": "system", "content": "당신은 소프트웨어 분석 및 분류 전문가입니다."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3
        )
        content = response.choices[0].message.content or ""
        lines = [line.strip() for line in content.splitlines() if ":" in line]

        def extract_value(prefix):
            for line in lines:
                if line.startswith(prefix):
                    parts = line.split(":", 1)
                    if len(parts) == 2:
                        return parts[1].strip()
            return "미분류"

        return {
            "category_large": extract_value("대분류"),
            "category_medium": extract_value("중분류"),
            "category_small": extract_value("소분류"),
        }
    except Exception as e:
        print(f"Error in classify_requirement_logic: {e}")
        return {
            "category_large": "Error",
            "category_medium": "Error",
            "category_small": "Error"
        }

# 작업 2: 난이도 평가
def generate_difficulty_prompt_text(description, detailed_description, module):
    return f"""
당신은 소프트웨어 요구사항의 기술적 구현 난이도를 분석하고 평가하는 **수십 년 경력의 베테랑 개발 팀장 또는 시스템 아키텍트**입니다. 제시된 요구사항의 **기술적 복잡성, 필요한 리서치 및 학습량, 구현에 필요한 공수, 외부 시스템과의 연동 복잡성, 테스트의 난해함, 그리고 잠재적인 리스크** 등을 종합적으로 고려하여 난이도를 '상', '중', '하' 중 하나로 매우 신중하고 일관성 있게 평가해야 합니다.

다음은 시스템에 대한 요구사항입니다:

[요구사항 설명]
{description}

[상세 설명]
{detailed_description}

[담당 모듈]
{module}

요구사항을 분석한 뒤, 다음의 **판단 가이드라인과 세부 평가 기준**을 참고하여 난이도를 평가하세요:

**[판단 가이드라인: 난이도에 영향을 미치는 주요 요소]**
1.  **요구사항의 명확성 및 구체성:** 요구사항이 모호하거나 해석의 여지가 많을수록 분석 및 설계 단계부터 어려움이 추가되어 난이도가 상승합니다.
2.  **기술적 생소함 및 복잡도:** 새로운 프로그래밍 언어, 프레임워크, 라이브러리, 알고리즘의 도입이 필요하거나, 기존에 다뤄보지 않은 매우 복잡한 기술적 구현이 요구될 경우 난이도가 높습니다.
3.  **시스템 연동의 범위 및 복잡도:** 연동해야 할 내부/외부 시스템의 수가 많거나, 연동 방식(API, 프로토콜 등)이 복잡하거나, 연동 대상 시스템의 문서화가 미흡하거나 기술 지원이 원활하지 않을 경우 난이도가 크게 상승합니다.
4.  **데이터 처리 및 마이그레이션의 복잡성:** 처리해야 할 데이터의 양이 매우 방대하거나, 데이터 구조가 복잡하거나, 기존 시스템과의 데이터 정합성 유지 및 마이그레이션 작업이 까다로울 경우 난이도가 높습니다.
5.  **기존 시스템에 대한 영향 (Side Effect):** 요구사항 구현으로 인해 기존 시스템의 다른 부분에 예상치 못한 영향을 미칠 가능성이 높고, 이로 인해 광범위한 테스트와 수정이 필요할 경우 난이도가 증가합니다.
6.  **테스트의 복잡성 및 용이성:** 단위 테스트, 통합 테스트, 시스템 테스트 시나리오가 복잡하거나, 테스트 환경 구축이 어렵거나, 테스트 데이터 생성이 까다로운 경우 난이도가 높습니다.
7.  **비기능적 요구사항의 달성 난이도:** 매우 높은 수준의 성능(응답 시간, 처리량), 보안(암호화, 접근 제어), 안정성, 확장성 등의 비기능적 요구사항을 만족시켜야 한다면 기술적 도전 과제가 많아져 난이도가 상승합니다.
8.  **유지보수성 고려:** 향후 유지보수가 용이하도록 코드를 구조화하고 문서화하는 데 추가적인 노력이 많이 필요할 것으로 예상되면 난이도에 반영될 수 있습니다.

**[난이도 평가 기준]**

* **상 (H, High):**
    * **판단 근거:** 위의 판단 가이드라인 중 **다수 항목에서 높은 수준의 복잡성 또는 불확실성**이 확인되거나, **특정 한두 요소가 프로젝트 일정에 심각한 지연을 초래할 만큼 매우 치명적인 기술적 장벽**을 포함하는 경우.
    * **특징적 상황:**
        * 핵심 아키텍처 변경 또는 검증되지 않은 신기술의 광범위한 도입/연구가 필요함.
        * 매우 복잡한 알고리즘 설계 및 구현, 또는 전례 없는 수준의 시스템 연동이 요구됨.
        * 요구사항의 불확실성이 극도로 높아 초기 분석/설계 단계에서부터 상당한 시간과 리서치, PoC(Proof of Concept)가 필요함.
        * 구현 실패의 리스크가 높거나, 성공하더라도 사전에 계획된 개발 일정을 현저히 초과할 가능성이 매우 농후함.
        * 해결을 위해 팀 내 최고 수준의 전문가 투입 또는 외부 전문 컨설팅이 필요할 수 있음.
    * **일정 영향:** 자체 개발 일정에 **심각한 차질(예: 주요 마일스톤의 상당한 지연, 예정된 리소스 초과 등)**을 초래할 정도의 매우 높은 노력과 시간이 필요하며, 별도의 핵심 과제로 관리되어야 하는 수준.

* **중 (M, Medium):**
    * **판단 근거:** 판단 가이드라인 중 **일부 항목에서 중간 정도의 복잡성**이 관찰되거나, 해결해야 할 몇 가지 기술적 고려사항 및 도전 과제가 명확히 존재하는 경우.
    * **특징적 상황:**
        * 익숙한 기술 스택을 기반으로 하지만 새로운 기능을 개발하거나 기존 기능에 대한 상당한 수정이 필요함.
        * 일부 복잡한 비즈니스 로직, 혹은 예측 가능한 범위 내의 기술적 문제 해결이 요구됨.
        * 내부 모듈 간의 복잡한 상호작용 또는 비교적 잘 정의된 외부 시스템과의 연동 작업이 포함됨.
        * 어느 정도의 분석/설계 시간이 필요하며, 구현 중 예상치 못한 이슈가 발생할 수 있으나 관리 가능한 수준.
    * **일정 영향:** 집중적인 노력과 계획적인 접근을 통해 **자체 개발 일정 내에 충분히 완수 가능**하나, 일정 내에서도 타이트하거나 약간의 도전이 따를 수 있는 수준.

* **하 (L, Low):**
    * **판단 근거:** 판단 가이드라인의 대부분 항목에서 복잡성이 낮거나, 기술적으로 **매우 명확하고 직접적인 해결 방법**이 존재하는 경우.
    * **특징적 상황:**
        * 기존 기능의 단순 버그 수정, UI 텍스트 변경, 경미한 디자인 조정, 이미 잘 구축된 컴포넌트의 재활용 또는 매우 간단한 로직의 추가.
        * 새로운 기술 학습이나 복잡한 분석/설계 과정이 거의 불필요하며, 구현 경로가 명확함.
        * 타 시스템과의 연동이 없거나 매우 단순하며, 테스트가 용이함.
    * **일정 영향:** 현재 진행 중인 다른 작업과 병행하거나, **자체 개발 일정의 일부분으로 특별한 부담 없이 충분히 흡수**되어 별도의 추가 공수 산정 없이 진행 가능한 수준.

아래와 같이 **정확히 이 형식**으로만 출력하세요 (불필요한 설명 없이):

난이도: <상|중|하>
"""

def get_difficulty_logic(description: str, detailed_description: str, module: str) -> str:
    prompt = generate_difficulty_prompt_text(description, detailed_description, module)
    try:
        response = client.chat.completions.create(
            model="gpt-4", # 또는 gpt-4-turbo, gpt-3.5-turbo 등 사용 가능한 모델
            messages=[
                {"role": "system", "content": "당신은 소프트웨어 분석 전문가입니다."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.4
        )
        content = response.choices[0].message.content or ""
        difficulty = next((line.split(":")[1].strip() for line in content.splitlines() if "난이도" in line), "중") # 기본값을 '중'으로 설정
        return difficulty
    except Exception as e:
        print(f"Error in get_difficulty_logic: {e}")
        return "Error"

# 작업 3: 중요도 평가
def generate_importance_prompt_text(description, detailed_description, module):
    return f"""
당신은 소프트웨어 요구사항을 분석하여 중요도를 판단하는 **매우 숙련되고 비판적인 시스템 분석가**입니다. 제시된 기준과 판단 가이드라인에 따라 각 요구사항의 중요도를 '상', '중', '하' 중 하나로 **극도로 신중하고 일관성 있게** 평가해야 합니다. **'상' 등급은 매우 제한적으로 사용되어야 함**을 명심하십시오.

다음은 시스템에 대한 요구사항입니다:

[요구사항 설명]
{description}

[상세 설명]
{detailed_description}

[담당 모듈]
{module}

요구사항을 분석한 뒤, 아래의 **세분화된 기준과 판단 가이드라인**에 따라 중요도를 평가하세요:

**[판단 가이드라인]**
1.  **가장 먼저 '상' (Critical)에 해당하는지 극도로 보수적으로 판단합니다.** 이 요구사항이 없으면 시스템 자체가 완전히 무너지거나 법적/보안적으로 회복 불가능한 치명적 문제가 발생하는지 자문하십시오. **대부분의 요구사항은 '상'에 해당하지 않을 가능성이 높습니다.**
2.  '상'이 아니라면, '중' (Important)에 해당하는지 검토합니다.
3.  '상'도 '중'도 아니라면 '하' (Useful)로 평가합니다.
4.  요구사항의 단어나 문구에 현혹되지 말고, **실제 시스템 전체에 미치는 파급 효과와 해당 요구사항 실패 시의 구체적인 결과를 기준으로 냉정하게 판단**하십시오. 모든 요구사항이 중요해 보일 수 있지만, 자원은 한정되어 있으므로 상대적인 중요도를 엄격히 구분해야 합니다.

**[중요도 평가 기준]**

* **상 (C, Critical):**
    * **판단 기준:** 해당 요구사항의 미구현이 **시스템 전체의 핵심 기능 마비, 서비스 불가능 상태 초래, 심각한 법적/규제적 문제 야기, 대규모 중요 데이터의 영구적 손실 또는 오염, 회복 불가능한 치명적 보안 사고 발생**과 같이 프로젝트의 존립을 위협하거나 시스템 전체의 실패를 의미하는 경우에만 해당합니다. **대체 수단이 전혀 없거나, 그 영향이 조직/서비스 전체에 즉각적이고 치명적인 경우**에만 극히 제한적으로 부여합니다.
    * **'상'이 아닌 경우 (예시):** 단순히 "필수적"이라고 언급되거나, 중요한 기능처럼 보이더라도, 위와 같은 수준의 치명적이고 즉각적인 결과로 이어지지 않는다면 '상'으로 평가해서는 안 됩니다. 예를 들어, 특정 기능의 부재가 큰 불편을 야기하지만 시스템의 다른 핵심 기능은 정상 동작한다면 '상'이 아닙니다.

* **중 (I, Important):**
    * **판단 기준:** 시스템의 기능적 완성도, 운영 효율성, 사용자 만족도에 **상당한 영향을 미치지만, 그것이 없다고 해서 시스템 전체가 즉시 마비되거나 사용 불가능 상태가 되지는 않는 경우**입니다. 미구현 시 서비스 중단까지는 아니지만, 주요 사용자의 큰 불편을 초래하거나, 기업의 수익/평판에 측정 가능한 부정적 영향을 미치거나, 핵심 업무 프로세스에 심각한 차질을 주는 경우 해당됩니다.
    * **'중'이 아닌 경우 (예시):** 사소한 불편함, 일부 제한된 사용자에게만 영향, 또는 있으면 좋지만 없어도 큰 지장이 없는 경우는 '중'이 아닙니다.

* **하 (U, Useful):**
    * **판단 기준:** 구현되면 유용하고 사용자 경험을 개선할 수 있지만, 미구현되어도 시스템의 핵심 기능, 안정성, 보안 및 주요 사용자 그룹의 전반적인 만족도에 **심각한 영향을 주지 않는 사항**입니다. 약간의 불편함이 있거나, 특정 소수의 사용자에게만 영향을 미치거나, 다른 기능으로 비교적 쉽게 대체 가능하거나, 장기적으로 고려할 만한 개선 사항인 경우 해당됩니다.

아래와 같이 **정확히 이 형식**으로만 출력하세요 (불필요한 설명 없이):

중요도: <상|중|하>
"""

def get_importance_logic(description: str, detailed_description: str, module: str) -> str:
    prompt = generate_importance_prompt_text(description, detailed_description, module)
    try:
        response = client.chat.completions.create(
            model="gpt-4", # 또는 gpt-4-turbo, gpt-3.5-turbo 등 사용 가능한 모델
            messages=[
                {"role": "system", "content": "당신은 소프트웨어 분석 전문가입니다."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.4
        )
        content = response.choices[0].message.content or ""
        importance = next((line.split(":")[1].strip() for line in content.splitlines() if "중요도" in line), "중") # 기본값을 '중'으로 설정
        return importance
    except Exception as e:
        print(f"Error in get_importance_logic: {e}")
        return "Error"

# --- LangGraph 부분 시작 ---
# langgraph 라이브러리가 설치되어 있어야 합니다: pip install langgraph
try:
    from langgraph.graph import StateGraph, END
except ImportError:
    print("langgraph 라이브러리가 설치되지 않았습니다. 'pip install langgraph'로 설치해주세요.")
    exit()

class RequirementAnalysisState(TypedDict, total=False):
    # Fields from your input JSON
    id: str
    type: str
    description: str
    detailed_description: str
    acceptance_criteria: str
    responsible_module: str
    parent_id: str
    source_pages: List[int]

    # Fields populated by LLM tasks
    # classification: Dict[str, str]  # 이 줄을 삭제합니다.
    category_large: str             # 다음 세 줄을 추가합니다.
    category_medium: str
    category_small: str
    difficulty: str
    importance: str

    # Final aggregated output for each item
    combined_results: Dict[str, Any]
# LangGraph 노드 정의
# 각 노드는 상태(state)를 입력으로 받고, 업데이트할 상태 부분을 담은 딕셔너리를 반환합니다.

# (node_classify_requirement, node_get_difficulty, node_get_importance는 병렬 실행 노드에서 직접 호출되므로 여기서는 생략 가능)

# 병렬 실행을 위한 통합 노드
# 이 노드 내에서 ThreadPoolExecutor를 사용하여 세 가지 작업을 병렬로 실행합니다.
def node_parallel_assessments(state: RequirementAnalysisState) -> Dict[str, Any]:
    description = state["description"]
    detailed_description = state["detailed_description"]
    module = state["responsible_module"]

    # results 딕셔너리는 각 로직 함수의 반환 값을 임시 저장합니다.
    # classify_requirement_logic의 반환 값은 여전히 딕셔너리입니다.
    classification_dict: Dict[str, str] = {} # 타입 힌트 추가
    difficulty_str: str = ""
    importance_str: str = ""

    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        future_classify = executor.submit(classify_requirement_logic, description, detailed_description, module)
        future_difficulty = executor.submit(get_difficulty_logic, description, detailed_description, module)
        future_importance = executor.submit(get_importance_logic, description, detailed_description, module)

        classification_dict = future_classify.result()
        difficulty_str = future_difficulty.result()
        importance_str = future_importance.result()

    # 반환하는 딕셔너리가 RequirementAnalysisState의 개별 키와 일치하도록 수정
    return {
        "category_large": classification_dict.get("category_large", "미분류"),
        "category_medium": classification_dict.get("category_medium", "미분류"),
        "category_small": classification_dict.get("category_small", "미분류"),
        "difficulty": difficulty_str,
        "importance": importance_str,
    }

# 모든 결과를 취합하는 노드
def node_combine_results(state: RequirementAnalysisState) -> Dict[str, Any]:

    # combined_data는 현재 state의 모든 항목 (입력 + LLM 결과)을 포함해야 합니다.
    # 'combined_results' 키 자체는 최종 상태 업데이트를 위한 것이므로 제외합니다.
    
    combined_data_for_output: Dict[str, Any] = {}
    for key, value in state.items():
        if key != "combined_results": # 'combined_results'는 이 노드가 채울 필드이므로 복사 대상에서 제외
            combined_data_for_output[key] = value
    
    # LLM 분석 결과 필드가 명시적으로 포함되었는지 확인 (이미 state.items()에 포함되어 있을 것임)
    # 이 부분은 state.items() 반복으로 이미 처리되므로 중복될 수 있으나,
    # 명시적으로 값을 가져오거나 기본값을 설정하고 싶다면 유지할 수 있습니다.
    # combined_data_for_output["classification"] = state.get("classification", {"Error": "Not processed"})
    # combined_data_for_output["difficulty"] = state.get("difficulty", "Error")
    # combined_data_for_output["importance"] = state.get("importance", "Error")

    # 이 노드는 상태의 'combined_results' 필드를 업데이트할 딕셔너리를 반환합니다.
    return {"combined_results": combined_data_for_output}

# 그래프 빌더 생성
workflow = StateGraph(RequirementAnalysisState)

# 노드 추가
workflow.add_node("parallel_processor", node_parallel_assessments)
workflow.add_node("final_combiner", node_combine_results) # 병렬 처리된 결과를 최종 정리

# 엣지 연결
workflow.set_entry_point("parallel_processor")
workflow.add_edge("parallel_processor", "final_combiner")
workflow.add_edge("final_combiner", END)


# 그래프 컴파일
app = workflow.compile()

In [49]:
# --- 새로운 메인 로직 ---
def load_requirements_from_json(filepath: str) -> List[Dict[str, str]]:
    """지정된 경로에서 JSON 파일을 로드하여 요구사항 목록을 반환합니다."""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
            if isinstance(data, list):
                return data
            else:
                print(f"Error: JSON 파일 ({filepath})이 리스트 형태가 아닙니다.")
                return []
    except FileNotFoundError:
        print(f"Error: JSON 파일을 찾을 수 없습니다 - {filepath}")
        return []
    except json.JSONDecodeError:
        print(f"Error: JSON 파일 디코딩 중 오류 발생 - {filepath}")
        return []
    except Exception as e:
        print(f"Error loading JSON file {filepath}: {e}")
        return []

def save_results_to_json(results: List[Dict[str, Any]], filepath: str):
    """결과 목록을 지정된 경로에 JSON 파일로 저장합니다."""
    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=2, ensure_ascii=False)
        print(f"\n결과가 성공적으로 {filepath} 파일에 저장되었습니다.")
    except Exception as e:
        print(f"Error saving results to JSON file {filepath}: {e}")

def main():
    """
    JSON 파일에서 요구사항을 로드하고, 각 요구사항을 처리한 후,
    모든 결과를 단일 JSON 파일로 저장합니다.
    """
    # 실제 API 사용 시 API 키 확인
    # 이 코드는 MockClient를 사용하므로 이 부분을 주석 처리하거나 실제 client 초기화 로직에 맞게 수정합니다.
    # if not isinstance(client, MockOpenAIClient) and not os.getenv("OPENAI_API_KEY"):
    #     print("Error: OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")
    #     print("실제 OpenAI API를 사용하려면 API 키를 설정해주세요.")
    #     print("스크립트를 종료합니다.")
    #     return

    input_json_path = "docs/test_lv3.json"
    output_json_path = "processed_requirements_output.json"

    requirements_to_process = load_requirements_from_json(input_json_path)

    if not requirements_to_process:
        print("처리할 요구사항이 없습니다. 스크립트를 종료합니다.")
        return

    all_combined_results = []
    total_requirements = len(requirements_to_process)

    print(f"\n총 {total_requirements}개의 요구사항 처리를 시작합니다...")

    for i, req_data in enumerate(requirements_to_process):
        print(f"\n[{i+1}/{total_requirements}] 처리 중: '{req_data.get('description', 'N/A')[:70]}...'")

        # LangGraph에 전달할 초기 상태 구성
        # req_data는 input JSON 파일의 각 객체입니다.
        inputs_for_graph: RequirementAnalysisState = {
            # Map all fields from req_data to the state
            "id": req_data.get("id"),
            "type": req_data.get("type"),
            "description": req_data.get("description", "내용 없음"), # Used by LLMs
            "detailed_description": req_data.get("detailed_description", "상세 내용 없음"), # Used by LLMs
            "acceptance_criteria": req_data.get("acceptance_criteria"),
            "responsible_module": req_data.get("responsible_module"), # Original module field
            "parent_id": req_data.get("parent_id"),
            "source_pages": req_data.get("source_pages"),
            
            # classification, difficulty, importance, combined_results는 그래프 내에서 채워집니다.
        }

        try:
            # LangGraph 실행
            final_state = app.invoke(inputs_for_graph)

            if final_state and "combined_results" in final_state:
                all_combined_results.append(final_state["combined_results"])
            else:
                print(f"    ⚠️ [{i+1}/{total_requirements}] 요구사항 처리 후 'combined_results'를 찾을 수 없습니다. Skipping.")
                # 에러 상황에 대한 대체 결과 추가 가능
                error_result = {
                    "input_description": inputs_for_graph["description"],
                    "error": "Processing failed or combined_results not found in final state.",
                    "details": final_state
                }
                all_combined_results.append(error_result)


        except Exception as e:
            print(f"    ❌ [{i+1}/{total_requirements}] 요구사항 처리 중 오류 발생: {e}")
            traceback.print_exc()
            # 오류 발생 시에도 입력 정보를 포함한 에러 메시지를 결과에 추가
            error_result = {
                "input_description": inputs_for_graph["description"],
                "input_detailed_description": inputs_for_graph["detailed_description"],
                "input_module": inputs_for_graph["module"],
                "error": str(e),
                "classification": "Error",
                "difficulty": "Error",
                "importance": "Error"
            }
            all_combined_results.append(error_result)

    # 모든 결과를 JSON 파일로 저장
    save_results_to_json(all_combined_results, output_json_path)

if __name__ == "__main__":
    print("스크립트 시작...")
    main()

스크립트 시작...

총 6개의 요구사항 처리를 시작합니다...

[1/6] 처리 중: 'UI/UX 디자인 가이드라인 개발...'

[2/6] 처리 중: '컴포넌트 라이브러리 구축...'

[3/6] 처리 중: '사용자 인터페이스 표준화...'

[4/6] 처리 중: '접근성 표준 준수...'

[5/6] 처리 중: '반응형 디자인 구현...'

[6/6] 처리 중: '사용자 피드백 수집 및 반영...'

결과가 성공적으로 processed_requirements_output.json 파일에 저장되었습니다.
